In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

import re

In [2]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [3]:
##############
# Parameters #
##############

min_gram = 1
max_gram = 3

# To create ours partitions, we must first know the years which will be the limits
limit_years = [2007, 2010, 2014]

# Ignore words that appear at a frequency less than tresh_freq in the corpus
tresh_freq = 0.8

# Number of clusters by partitions
nb_clusters = 5


In [4]:
# Datas preprocessing methods.

# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw

# Based on ration of french and english stopwords
def isEnglish(article):
    total_fsw = len([x for x in article.split() if x in sourceFST])
    total_esw = len([x for x in article.split() if x in stopwords.words('english')])
    ratio = 100
    if total_fsw != 0:
        ratio = total_esw/total_fsw
    return ratio > 1 and total_esw > 3

def lemmatize(article):
    artiregex = re.sub(" [A-z][A-z] ", " ", article) # word of length < 2
    artiregex = artiregex.lower()
    artiregex = re.sub("(é|è|ê)", "e", artiregex)
    output = []
    outPonc = artiregex.translate(artiregex.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST and [x for x in token.lemma_ if x not in "0123456789"] != []:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [5]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [6]:
# Let's process our corpus, and determine a limit to split it in partitions

# usable[] correspond to our corpus processed
# limits[] let us know when to delimit partitions
limits = []
usable = []

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    #if not null, empty, or whatever (so if there is a abstract):
    if not isinstance(data['abstract'][i], float) and not isEnglish(data['abstract'][i]):
        text = data['abstract'][i]
        if not isinstance(data['title'][i], float):
            text += " "+data['title'][i]

        numArti+=1
        usable.append(stemmer.stem(lemmatize(text)))
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti)
limits.append(numArti)


In [7]:
# Display pre-processed datas

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram), max_df=tresh_freq)
tfidf = vectorizer.fit_transform(usable)

print("nombre d'articles =", len(usable))
print("nombre de mots =", len(tfidf.toarray()[0]))
print("limits =", limits)

nombre d'articles = 991
nombre de mots = 133088
limits = [223, 468, 694, 991]


In [8]:
# Creation of partitions_tfidf[], which give us the TFIDF of each cluster of each partition
# partitions_tfidf[num_partition][num_doc][num_word]
# Beware, num_doc can't be equals to 1091 (max). You have partitions, so every doc aren't in every partitions
# num_word can be found via vectorizer.get_feature_name()
partitions_tfidf = []
beg = 0
for l in limits:
    last = l
    partitions_tfidf.append([list(x) for x in list(tfidf.toarray())[beg:last]])
    beg = l

In [9]:
size = len(partitions_tfidf[0]) + len(partitions_tfidf[1]) + len(partitions_tfidf[2]) + len(partitions_tfidf[3])
size

991

In [30]:
len(partitions_tfidf[1])

245

In [38]:
# Applying KMeans on tfidf
# the labels_ give assignment of doc to the cluster number

In [39]:
# doc_clustering is a dictionnary 
# it looks like -> { doc_number : [partition_number, cluster_number] }
# This is used to reassign doc number to their respective partition and and cluster

doc_clustering = {}
numDoc = 0
for i in range(0, len(limits)):
    dash = km.fit(partitions_tfidf[i])
    previousBound = 0
    if i > 0:
        previousBound = limits[i-1]
    for numDocItern in range(0, limits[i]-previousBound):
        doc_clustering[numDoc] = [i, dash.labels_[numDocItern]]
        numDoc+=1

In [34]:
doc_clustering

{0: [0, 1],
 1: [0, 3],
 2: [0, 4],
 3: [0, 0],
 4: [0, 2],
 5: [0, 0],
 6: [0, 3],
 7: [0, 4],
 8: [0, 3],
 9: [0, 1],
 10: [0, 3],
 11: [0, 1],
 12: [0, 4],
 13: [0, 1],
 14: [0, 2],
 15: [0, 0],
 16: [0, 0],
 17: [0, 2],
 18: [0, 2],
 19: [0, 4],
 20: [0, 1],
 21: [0, 1],
 22: [0, 1],
 23: [0, 3],
 24: [0, 0],
 25: [0, 1],
 26: [0, 2],
 27: [0, 4],
 28: [0, 4],
 29: [0, 4],
 30: [0, 1],
 31: [0, 1],
 32: [0, 1],
 33: [0, 0],
 34: [0, 2],
 35: [0, 3],
 36: [0, 1],
 37: [0, 3],
 38: [0, 2],
 39: [0, 3],
 40: [0, 3],
 41: [0, 1],
 42: [0, 4],
 43: [0, 4],
 44: [0, 1],
 45: [0, 1],
 46: [0, 1],
 47: [0, 3],
 48: [0, 0],
 49: [0, 2],
 50: [0, 3],
 51: [0, 4],
 52: [0, 3],
 53: [0, 0],
 54: [0, 4],
 55: [0, 1],
 56: [0, 1],
 57: [0, 2],
 58: [0, 2],
 59: [0, 3],
 60: [0, 3],
 61: [0, 4],
 62: [0, 3],
 63: [0, 0],
 64: [0, 1],
 65: [0, 3],
 66: [0, 2],
 67: [0, 1],
 68: [0, 4],
 69: [0, 3],
 70: [0, 0],
 71: [0, 2],
 72: [0, 3],
 73: [0, 2],
 74: [0, 2],
 75: [0, 2],
 76: [0, 2],
 77: [0, 

In [35]:
# Allows to get list of documents number
# return [dou numbers]
# params : partition_number , cluster number
def get_doc(part, clust):
    docs = []
    for i in range(0,len(doc_clustering)):
        if doc_clustering[i][0] == part and doc_clustering[i][1] == clust:
            docs.append(i)
    return docs

In [36]:
# Get the partitions variable
# Here partitions[part][cluster] = list of docs numbe
partitions = []
for i in range(0, len(limits)):
    clusters = []
    for j in range(0, nb_clusters):
        clusters.append(get_doc(i,j))
    partitions.append(clusters)

In [37]:
partitions

[[[3,
   5,
   15,
   16,
   24,
   33,
   48,
   53,
   63,
   70,
   81,
   82,
   101,
   106,
   111,
   116,
   129,
   136,
   143,
   146,
   147,
   165,
   168,
   172,
   174,
   180,
   183,
   195,
   208],
  [0,
   9,
   11,
   13,
   20,
   21,
   22,
   25,
   30,
   31,
   32,
   36,
   41,
   44,
   45,
   46,
   55,
   56,
   64,
   67,
   80,
   83,
   87,
   90,
   92,
   93,
   98,
   99,
   103,
   109,
   114,
   117,
   119,
   120,
   121,
   122,
   123,
   127,
   130,
   134,
   140,
   144,
   149,
   152,
   153,
   157,
   158,
   161,
   176,
   189,
   191,
   197,
   201,
   203,
   205,
   210,
   213,
   214,
   215,
   216,
   219,
   221],
  [4,
   14,
   17,
   18,
   26,
   34,
   38,
   49,
   57,
   58,
   66,
   71,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   105,
   113,
   125,
   126,
   128,
   137,
   138,
   156,
   162,
   166,
   171,
   178,
   182,
   185,
   192,
   199,
   200,
   204,
   222],
  [1,
   6,
   8,
   10,
   2

In [ ]:
# example of output for doc_clustering
# doc 465 is in cluster 1 of the partition 1
# doc 154 is in cluster 2 of the partition 0

print(doc_clustering[465])
print(doc_clustering[154])
print()

# Here, just count docs number by cluster.
print(Counter(cluster_partition[0]))
print(Counter(cluster_partition[1]))
print(Counter(cluster_partition[2]))
print(Counter(cluster_partition[3]))

###### A quoi sert le reste en dessous là ? 

In [ ]:
# num_partition = 0
# for partition in partitions_tfidf:
#     if (num_partition > 2):
#         break
#     print("Partition numéro:",num_partition)
#     num_partition+=1
#     i=0
#     for cluster_tfidf in partition:
#         if (i<3):
#             print("cluster number ", i)
#             i+=1
#             print(cluster_tfidf)


In [ ]:
# # Let's create ours partitions
# partitions = []

# for part in partitions_tfidf:
#     # Let's create ours clusters
#     clusters = []
#     kmeans = KMeans(n_clusters=nb_clusters, random_state=0).fit(part)
#     partitions.append(clusters)
    
# # TODO: it would be nice to know how many articles are in no cluster anymore

In [ ]:
# # Display clusters 3 of partition 0 
# len(partitions[0])

In [ ]:
# nb_labels_by_cluster = 5

# # Let's labelize our clusters
# # For this, we will use the tfidf matrix

# vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram))
# tfidf = vectorizer.fit_transform(usable)

# # We can access the value in the tfidf using:
# #tfidf.toarray().item(num_doc, num_word)
# # To know the number of the word searched, we will use:
# #vectorizer.vocabulary_[word]

# # take less than 8h to compute x)
# labels = []
# for clusters in partitions:
#     l = []
#     for clus in clusters:
#         first_arti = True
#         for article in clus:
#             link = abs(clus[article])
#             if first_arti:
#                 coef_list = (tfidf.toarray()[article] * link)
#                 first = False
#             else:
#                 # the more an article have a high coeficient, the more he is implied in the labeling step
#                 coef_list += (tfidf.toarray()[article] * link)
#         # Now we have coef_list filled by every coeficient in the multiple tfidf
#         # Let's find the best ones, to finally get the labels
#         res = dict(zip(vectorizer.get_feature_names(), coef_list))

#         l.append(Counter(res).most_common(nb_labels_by_cluster))
#     labels.append(l)

# # TODO: on observe beaucoup de labels identiques entre deux clusters
# # Je pense que c'est parce que l'on a trop de clusters, mais j'aimerais en être sûr

In [ ]:
# # Display labels
# # labels is composed by an array for each partition
# labels